In [2]:
import os
import numpy
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.image as mpimg
from itertools import cycle
import seaborn as sns
import statsmodels.api as sm 
from scipy.interpolate import interp1d
from prophet import Prophet

pd.set_option('display.max_columns', 50)
plt.style.use('bmh')
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])

c:\Users\fuyuk\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [3]:
%%time
oj = os.path.join
INPUT_DIR = '../input/m5-forecasting-accuracy/'
train = pd.read_csv(oj(INPUT_DIR, 'trainset.csv'))
price = pd.read_csv(oj(INPUT_DIR, 'sell_prices.csv'))
calendar = pd.read_csv(oj(INPUT_DIR, 'calendar.csv'))

sample_submit = pd.read_csv(oj(INPUT_DIR, 'submit.csv'))

CPU times: total: 3.47 s
Wall time: 3.49 s


In [12]:
print(calendar['event_type_1'].unique())

[nan 'Sporting' 'Cultural' 'National' 'Religious']


In [6]:
print(calendar['event_name_2'].unique())

[nan 'Easter' 'Cinco De Mayo' 'OrthodoxEaster' "Father's day"]


In [7]:
print(calendar['event_type_2'].unique())

[nan 'Cultural' 'Religious']


In [8]:
print(price.head())
print(price.shape)

  store_id        item_id  wm_yr_wk  sell_price
0     CA_1  HOBBIES_1_001     11325        9.58
1     CA_1  HOBBIES_1_001     11326        9.58
2     CA_1  HOBBIES_1_001     11327        8.26
3     CA_1  HOBBIES_1_001     11328        8.26
4     CA_1  HOBBIES_1_001     11329        8.26
(6841121, 4)


In [5]:
print(train.shape)

(30490, 1933)


In [10]:
for i in range(train.shape[0]):
    df = pd.DataFrame(calendar[['date', 'd', 'wm_yr_wk']])
    df_train =  df.loc[0:1926]
    df_test = df.loc[1927:]
    #print(df_train)
    #print(df_test)
    df_train['y'] = train.loc[i,'d_1':].values
    print(df_train)
    df_train = pd.merge(df_train, price[(price['store_id'] == train.loc[i,'store_id']) & (price['item_id'] == train.loc[0,'item_id'])][['wm_yr_wk', 'sell_price']], how='left', on=['wm_yr_wk'])
    
    df_train = df_train.rename(columns={'date': 'ds'})
    #df_train = df_train.drop(["price"], axis=1)
    print(df_train)
    df_train = df_train.fillna({"sell_price":-1})
    print("-------------------")
    print(df_train)
    print("-------------------")
    
    model = Prophet()
    model.add_regressor('sell_price')
    model.fit(df_train)
    future = model.make_future_dataframe(periods=14)
    print(future)
    future = pd.merge(future, price[(price['store_id'] == train.loc[i,'store_id']) & (price['item_id'] == train.loc[0,'item_id'])][['wm_yr_wk', 'sell_price']], how='left')
    #future["sell_price"] = df_train["sell_price"]
    print(future)
    forecast = model.predict(future)
    print(forecast.tail(14))
    model.plot(forecast.tail(14))
    plt.show()
    #df_train.to_csv('df_train' + str(i) + '.csv', index=False)

C:\Users\fuyuk\AppData\Local\Temp\ipykernel_7528\666220853.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['y'] = train.loc[i,'d_1':].values


            date       d  wm_yr_wk  y
0     2011-01-29     d_1     11101  0
1     2011-01-30     d_2     11101  0
2     2011-01-31     d_3     11101  0
3     2011-02-01     d_4     11101  0
4     2011-02-02     d_5     11101  0
...          ...     ...       ... ..
1922  2016-05-04  d_1923     11614  1
1923  2016-05-05  d_1924     11614  1
1924  2016-05-06  d_1925     11614  0
1925  2016-05-07  d_1926     11615  2
1926  2016-05-08  d_1927     11615  1

[1927 rows x 4 columns]
              ds       d  wm_yr_wk  y  sell_price
0     2011-01-29     d_1     11101  0         NaN
1     2011-01-30     d_2     11101  0         NaN
2     2011-01-31     d_3     11101  0         NaN
3     2011-02-01     d_4     11101  0         NaN
4     2011-02-02     d_5     11101  0         NaN
...          ...     ...       ... ..         ...
1922  2016-05-04  d_1923     11614  1        8.38
1923  2016-05-05  d_1924     11614  1        8.38
1924  2016-05-06  d_1925     11614  0        8.38
1925  2016-05-07  d

16:44:11 - cmdstanpy - INFO - Chain [1] start processing
16:44:11 - cmdstanpy - INFO - Chain [1] done processing


             ds
0    2011-01-29
1    2011-01-30
2    2011-01-31
3    2011-02-01
4    2011-02-02
...         ...
1936 2016-05-18
1937 2016-05-19
1938 2016-05-20
1939 2016-05-21
1940 2016-05-22

[1941 rows x 1 columns]


KeyError: 'ds'